In [ ]:
import os

# Prevent OpenMP runtime conflicts (libomp vs libiomp) that crash the kernel
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Keep thread counts modest to reduce chance of runtime contention on Windows
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"


In [3]:
# ================= CONFIGURATION =================
INPUT_FOLDER = 'data/images_shift'
OUTPUT_FOLDER = 'output/shadow_masks'

In [ ]:
import cv2
import numpy as np
import glob
import os
import torch
from ultralytics import YOLO

# 1. SHADOW SETTINGS
RATIO_THRESHOLD = 0.65
ABSOLUTE_DARK_THRESHOLD = 40

# 2. PERSON HANDLING
USE_AI = True
AI_CONFIDENCE = 0.3
CONTEXT_RING_SIZE = 20
FILL_THRESHOLD = 0.5

# 3. CLEANUP
SKY_RATIO = 0.45
MIN_SHADOW_SIZE = 200 # Minimum pixel area to keep
# =================================================

# Pre-calculate kernels to save time
KERNEL_DILATE = np.ones((5,5), np.uint8)
KERNEL_MORPH = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
KERNEL_RING = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (CONTEXT_RING_SIZE, CONTEXT_RING_SIZE))

def create_background_model(files):
    print("Generating Background Model...")
    first_img = cv2.imread(files[0])
    max_buffer = np.zeros_like(first_img)

    for f in files:
        img = cv2.imread(f)
        if img is None: continue
        max_buffer = np.maximum(max_buffer, img)

    return max_buffer

def get_person_mask(img, model):
    h, w = img.shape[:2]
    person_mask = np.zeros((h, w), dtype=np.uint8)

    # Force GPU usage with device=0
    results = model(img, classes=[0], verbose=False, conf=AI_CONFIDENCE, device=0)

    if results[0].masks is not None:
        masks = results[0].masks.data.cpu().numpy()
        for mask in masks:
            mask_resized = cv2.resize(mask, (w, h))
            person_mask = np.maximum(person_mask, (mask_resized * 255).astype(np.uint8))

    person_mask = cv2.dilate(person_mask, KERNEL_DILATE, iterations=1)
    return person_mask

def segment_shadows_raw(img, bg_model):
    # Convert to float32 for faster vectorized division
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY).astype(np.float32)
    gray_bg = cv2.cvtColor(bg_model, cv2.COLOR_BGR2GRAY).astype(np.float32)

    safe_bg = gray_bg.copy()
    safe_bg[safe_bg == 0] = 1.0

    # Vectorized operations
    mask_ratio = (gray_img / safe_bg) < RATIO_THRESHOLD
    mask_absolute = gray_img < ABSOLUTE_DARK_THRESHOLD

    return np.logical_or(mask_ratio, mask_absolute).astype(np.uint8) * 255

def fill_person_holes(shadow_mask, person_mask):
    final_mask = shadow_mask.copy()

    if person_mask is None or np.count_nonzero(person_mask) == 0:
        return final_mask

    contours, _ = cv2.findContours(person_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop is unavoidable here for context-aware logic, 
    # but efficient OpenCV calls minimize impact
    for cnt in contours:
        single_person_mask = np.zeros_like(shadow_mask)
        cv2.drawContours(single_person_mask, [cnt], -1, 255, -1)

        dilated_person = cv2.dilate(single_person_mask, KERNEL_RING)
        ring_mask = cv2.subtract(dilated_person, single_person_mask)

        shadow_pixels_in_ring = cv2.bitwise_and(ring_mask, shadow_mask)

        total_ring_pixels = cv2.countNonZero(ring_mask)
        if total_ring_pixels == 0: continue
        
        shadow_ring_pixels = cv2.countNonZero(shadow_pixels_in_ring)

        if (shadow_ring_pixels / total_ring_pixels) > FILL_THRESHOLD:
            final_mask = cv2.bitwise_or(final_mask, single_person_mask)
        else:
            final_mask = cv2.bitwise_and(final_mask, cv2.bitwise_not(single_person_mask))

    return final_mask

def post_process_fast(mask, h):
    """
    Optimized cleanup using Vectorized NumPy operations.
    """
    # 1. Mask Sky
    sky_limit = int(h * SKY_RATIO)
    mask[:sky_limit, :] = 0

    # 2. Clean Noise
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, KERNEL_MORPH)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, KERNEL_MORPH)

    # 3. Filter Small Regions (VECTORIZED - HIGH SPEED)
    if MIN_SHADOW_SIZE > 0:
        # Get components
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)

        # stats[:, 4] is the Area column
        sizes = stats[:, cv2.CC_STAT_AREA]

        # Identify small blobs (True = remove, False = keep)
        # We must make sure Background (index 0) is False, or we'll delete the canvas
        small_blobs = sizes < MIN_SHADOW_SIZE
        small_blobs[0] = False 

        # This line is the magic speedup.
        # "labels" maps every pixel to an ID.
        # "small_blobs" maps every ID to True/False.
        # "small_blobs[labels]" generates a boolean mask for the whole image in one go.
        mask[small_blobs[labels]] = 0

    return mask

# ================= MAIN LOOP =================
if __name__ == "__main__":
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    files = sorted(glob.glob(os.path.join(INPUT_FOLDER, "*.jpg")))
    if not files:
        print("No images found.")
        exit()

    # 1. Load AI (Check for GPU)
    model = None
    if USE_AI:
        device = 0 if torch.cuda.is_available() else 'cpu'
        print(f"Loading YOLOv8 on {device}...")
        model = YOLO('yolov8n-seg.pt')

    # 2. Background Model
    bg_model = create_background_model(files)
    h, w = bg_model.shape[:2]
    cv2.imwrite(os.path.join(OUTPUT_FOLDER, "background_ref.jpg"), bg_model)

    print(f"Processing {len(files)} images...")

    for f in files:
        base_name = os.path.basename(f)
        img = cv2.imread(f)

        # A. Get Person Mask
        p_mask = None
        if USE_AI:
            p_mask = get_person_mask(img, model)

        # B. Get Raw Shadows
        raw_shadows = segment_shadows_raw(img, bg_model)

        # C. Remove People
        if p_mask is not None:
            shadows_no_people = cv2.bitwise_and(raw_shadows, cv2.bitwise_not(p_mask))
        else:
            shadows_no_people = raw_shadows

        # D. Intelligent Fill
        if p_mask is not None:
            final_mask = fill_person_holes(shadows_no_people, p_mask)
        else:
            final_mask = shadows_no_people

        # E. Fast Post Process
        final_mask = post_process_fast(final_mask, h)

        cv2.imwrite(os.path.join(OUTPUT_FOLDER, f"mask_{base_name}"), final_mask)
        print(f"Saved: mask_{base_name}")

    print("Processing Complete.")

Loading YOLOv8 on 0...
Generating Background Model...
Processing 36 images...
Saved: mask_IMG_20250906_110448_00_008.jpg
Saved: mask_IMG_20250906_111350_00_016.jpg
Saved: mask_IMG_20250906_112248_00_024.jpg
Saved: mask_IMG_20250906_113147_00_032.jpg
Saved: mask_IMG_20250906_114044_00_040.jpg
Saved: mask_IMG_20250906_114942_00_048.jpg
Saved: mask_IMG_20250906_115841_00_056.jpg
Saved: mask_IMG_20250906_120744_00_064.jpg
Saved: mask_IMG_20250906_121643_00_072.jpg
Saved: mask_IMG_20250906_122542_00_080.jpg
Saved: mask_IMG_20250906_123441_00_088.jpg
Saved: mask_IMG_20250906_124340_00_096.jpg
Saved: mask_IMG_20250906_125238_00_104.jpg
Saved: mask_IMG_20250906_130139_00_112.jpg
Saved: mask_IMG_20250906_131039_00_120.jpg
Saved: mask_IMG_20250906_131937_00_128.jpg
Saved: mask_IMG_20250906_132836_00_136.jpg
Saved: mask_IMG_20250906_160131_00_143.jpg
Saved: mask_IMG_20250906_161032_00_151.jpg
Saved: mask_IMG_20250906_161930_00_159.jpg
Saved: mask_IMG_20250906_162828_00_167.jpg
Saved: mask_IMG_202

: 

In [1]:
import matplotlib.pyplot as plt
import cv2
import os
import glob
import numpy as np

# ================= CONFIGURATION =================
# INPUT_FOLDER = 'images'     # Folder with original images
# OUTPUT_FOLDER = 'output_masks_people'    # Folder with generated masks
NUM_SAMPLES = 36                   # How many pairs to display
# =================================================

def display_results(input_dir, output_dir, limit=5, resize_factor=1.0):
    # Get list of original images
    input_files = sorted(glob.glob(os.path.join(input_dir, "*")))

    # Filter for image extensions only
    input_files = [f for f in input_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not input_files:
        print("No images found to display.")
        return

    # Loop through the desired number of samples
    for i, file_path in enumerate(input_files[:limit]):
        filename = os.path.basename(file_path)

        # Construct path to corresponding mask
        # (Assuming the naming convention from previous script: 'mask_filename')
        mask_path = os.path.join(output_dir, f"mask_{filename}")

        if not os.path.exists(mask_path):
            print(f"Mask not found for: {filename}")
            continue

        # Load images
        # 1. Original: Read as BGR, Convert to RGB for Matplotlib
        original = cv2.imread(file_path)
        original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)

        # 2. Mask: Read as Grayscale
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Resize images if resize_factor is less than 1.0
        if resize_factor < 1.0:
            h, w = original.shape[:2]
            original = cv2.resize(original, (int(w * resize_factor), int(h * resize_factor)), interpolation=cv2.INTER_AREA)
            mask = cv2.resize(mask, (int(w * resize_factor), int(h * resize_factor)), interpolation=cv2.INTER_NEAREST)

        # Plotting
        fig, axes = plt.subplots(1, 2, figsize=(20, 5)) # Wide layout for 360 images

        # Original Image
        axes[0].imshow(original)
        axes[0].set_title(f"Original: {filename}")
        axes[0].axis('off')

        # Generated Mask
        axes[1].imshow(mask, cmap='gray')
        axes[1].set_title("Generated Shadow Mask")
        axes[1].axis('off')

        plt.tight_layout()
        plt.show()

# Run the display
display_results(INPUT_FOLDER, OUTPUT_FOLDER, limit=NUM_SAMPLES, resize_factor=0.1)

NameError: name 'INPUT_FOLDER' is not defined

In [3]:
from panogeo.mapplot import mapping_shadow

saved = mapping_shadow(
    mask_paths="output/shadow_masks/*.jpg",           # list | dir | glob | file path
    calibration_npz="output/calibration_cam2enu.npz",
    out_dir="output/shadow_maps",
    provider="japan_gsi_air",                      # e.g., "carto", "osm", "japan_gsi_air"
    width_m=50.0,
    height_m=50.0,
    cell_size_m=1.0,                               # surface mesh resolution
    grass_mask_path=r"C:\Users\kunih\OneDrive\00_Codes\python\panogeo\output\grass_ground_mask.png",
    dem_path=None,                                 # or DEM path
    alpha=0.6,
    color="#000000",
    dpi=150,
    pixel_stride=1,
    stamp_timestamp=True
)

TypeError: '<=' not supported between instances of 'int' and 'NoneType'